<a href="https://colab.research.google.com/github/ganeshmb96/OpenCV/blob/master/Predicting_Age_and_Gender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Installing the dependancies
!pip install pafy   #tool used to retrieve youtube content and metadata
!pip install youtube_dl 

     |████████████████████████████████| 1.8MB 6.7MB/s 


In [2]:
#Example use case of using PAFY
import pafy

url = 'https://www.youtube.com/watch?v=c07IsbSNqfI&feature=youtu.be'
vpafy = pafy.new(url)
print(vpafy.title)
print(vpafy.rating)
print(vpafy.viewcount)
print(vpafy.author)
print(vpafy.length)
print(vpafy.description)

Testing file uploads with Postman (multipart/form-data)
4.7124181
36496
Valentin Despa
1688
💥 Want to learn more about Postman? Check out my Postman online course. 

Get it for only $9.99 (limited supply!)
http://bit.ly/2UmrlAC

___

// A B O U T  T H I S  V I D E O

I will show you how to debug an upload script and demonstrate it with a tool called Postman that can make requests encoded as "multipart/form-data" so that you can send also a file.

After this, we will go even further and write tests and begin automating the process.

Here is the Git repository containing the files used in this tutorial:
https://github.com/vdespa/postman-testing-file-uploads

___

// I  H A V E  A  Q U E S T I O N!

I do my best to answer all comments here on YouTube but I cannot make any guarantees.

If you have a question, it is best to ask your question on the Postman User Group on Facebook or on the Postman Community (links below). 

If you have purchased the Postman online course, please use the Q&A 

**Steps to follow :**
  1)Get the video URL from YouTube. 
  2)Face detection with Haar cascades 
  3)Gender Recognition with CNN 
Age Recognition with CNN

> Age Recognition with CNN


  1.Get a video URL from YouTube:
   Get the Youtube video URL and try to get the attributes of the video using pafy as explained above.
2. Face detection with Haar cascades :
This is a part most of us at least have heard of. OpenCV/JavaCV provide direct methods to import Haar-cascades and use them to detect faces. I will not be explaining this part in deep. You guys can refer to my previous article to know more about face detection using OpenCV.
3. Gender Recognition with CNN:
Gender recognition using OpenCV's fisherfaces implementation is quite popular and some of you may have tried or read about it also. But, in this example, I will be using a different approach to recognize gender. This method was introduced by two Israel researchers, Gil Levi and Tal Hassner in 2015. I have used the CNN models trained by them in this example. We are going to use the OpenCV’s dnn package which stands for “Deep Neural Networks”.
In the dnn package, OpenCV has provided a class called Net which can be used to populate a neural network. Furthermore, these packages support importing neural network models from well known deep learning frameworks like caffe, tensorflow and torch. The researchers I had mentioned above have published their CNN models as caffe models. Therefore, we will be using the CaffeImporter import that model into our application.

In [12]:
import cv2
import numpy as np
import pafy
#url of the video to predict Age and gender
url = 'https://www.youtube.com/watch?v=c07IsbSNqfI&feature=youtu.be'
vPafy = pafy.new(url)
play = vPafy.getbest(preftype="mp4") 
cap = cv2.VideoCapture(play.url)

cap.set(3, 480) #set width of the frame
cap.set(4, 640) #set height of the frame
MODEL_MEAN_VALUES = (78.4263377603, 87.7689143744, 114.895847746)
age_list = ['(0, 2)', '(4, 6)', '(8, 12)', '(15, 20)', '(25, 32)', '(38, 43)', '(48, 53)', '(60, 100)']
gender_list = ['Male', 'Female']

def load_caffe_models():
  age_net = cv2.dnn.readNetFromCaffe('deploy_age.prototxt', 'age_net.caffemodel')
  gender_net = cv2.dnn.readNetFromCaffe('deploy_gender.prototxt', 'gender_net.caffemodel')
  return(age_net, gender_net)

def video_detector(age_net, gender_net):
  font = cv2.FONT_HERSHEY_SIMPLEX
  while True:
    ret, image = cap.read()
       
  face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_alt.xml')
  gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  faces = face_cascade.detectMultiScale(gray, 1.1, 5)
  if(len(faces)>0):
    print("Found {} faces".format(str(len(faces))))
  for (x, y, w, h )in faces:
    cv2.rectangle(image, (x, y), (x+w, y+h), (255, 255, 0), 2)
#Get Face 
    face_img = image[y:y+h, h:h+w].copy()
    blob = cv2.dnn.blobFromImage(face_img, 1, (227, 227), MODEL_MEAN_VALUES, swapRB=False)
#Predict Gender
    gender_net.setInput(blob)
    gender_preds = gender_net.forward()
    gender = gender_list[gender_preds[0].argmax()]
    print("Gender : " + gender)
#Predict Age
    age_net.setInput(blob)
    age_preds = age_net.forward()
    age = age_list[age_preds[0].argmax()]
    print("Age Range: " + age)

    overlay_text = "%s %s" % (gender, age)
    cv2.putText(image, overlay_text, (x, y), font, 1, (255, 255, 255), 2, cv2.LINE_AA)
    cv2.imshow('frame', image)  
#0xFF is a hexadecimal constant which is 11111111 in binary.
    if cv2.waitKey(1) & 0xFF == ord('q'): 
      break
    
    
if __name__ == "__main__":
  age_net, gender_net = load_caffe_models()
  video_detector(age_net, gender_net)

error: ignored